<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [118]:
import pandas as pd
import psycopg2
import sqlalchemy

In [119]:
f = 'connect.yaml' # файл с параметрами подключения
d = {k:v.strip() for k, v in (l.split(':') for l in open(f))} # считывание параметров из файла
conn_str = f"postgresql+psycopg2://{d['user']}:{d['password']}@{d['host']}:{d['port']}/{d['dbname']}" # строка для подключения к БД

In [120]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
) # подключение к БД с явным указанием параметров (применено для юнита 6, т.к. метод с чтением параметров из файла вызывал ошибку)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [121]:
# текст запроса
query_3_1 = f'''select
                    count(*) "Количество вакансий в БД"
                from public.vacancies
             '''

In [122]:
# результат запроса
df_vac = pd.read_sql_query(query_3_1, conn_str)
display(df_vac)

,Количество вакансий в БД
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [123]:
# текст запроса
query_3_2 = f'''select
                    count(*) "Количество работодателей в БД"
                from public.employers
             '''

In [124]:
# результат запроса
df_emp = pd.read_sql_query(query_3_2, conn_str)
display(df_emp)

,Количество работодателей в БД
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [125]:
# текст запроса
query_3_3 = f'''select
                    count(*) "Количество регионов в БД"
                from public.areas
             '''

In [126]:
# результат запроса
df_reg = pd.read_sql_query(query_3_3, conn_str)
display(df_reg)

,Количество регионов в БД
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [127]:
# текст запроса
query_3_4 = f'''select
                    count(*) "Количество сфер деятельности в БД"
                from public.industries
             '''

In [128]:
# результат запроса
df_ind = pd.read_sql_query(query_3_4, conn_str)
display(df_ind)

,Количество сфер деятельности в БД
0,294


***

In [129]:
display(f"Среднее количество вакансий на 1 работодателя: {round((df_vac.iloc[0, 0] / df_emp.iloc[0, 0]), 2)}")
display(f"Среднее количество вакансий на 1 регион: {round((df_vac.iloc[0, 0] / df_reg.iloc[0, 0]), 2)}")
display(f"Среднее количество вакансий на 1 cферу деятельности: {round((df_vac.iloc[0, 0] / df_ind.iloc[0, 0]), 2)}")

'Среднее количество вакансий на 1 работодателя: 2.09'

'Среднее количество вакансий на 1 регион: 36.12'

'Среднее количество вакансий на 1 cферу деятельности: 167.34'

Выводы:

В БД содержится достаточное количество вакансий для анализа. Количество вакансий превышает количество работодателей более чем в 2 раза. В среднем на регион и сферу деятельности работодателя приходится по 36 и 167 вакансий соответственно, что позволяет проводить дельнейший анализ по отдельным критериям.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [130]:
# текст запроса
query_4_1 = f'''select
                    a.name area,
                    count(*) cnt
                from public.vacancies v
                join public.areas a on v.area_id = a.id /*соединение таблиц vacancies и areas по id региона*/
                group by area /*группировка по названию региона*/
                order by cnt desc /*сортировка по убыванию количества вакансий*/
                limit 5
             '''

In [131]:
# результат запроса
df = pd.read_sql_query(query_4_1, conn_str)
display(df)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [132]:
# текст запроса
query_4_2 = f'''select
                    count(*) "Количество вакансий с з/п"
                from public.vacancies
                where salary_from is not null or salary_to is not null /*условие наличия указанного значения зарплаты*/
             '''

In [133]:
# результат запроса
df = pd.read_sql_query(query_4_2, conn_str)
display(df)

,Количество вакансий с з/п
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [134]:
# текст запроса
query_4_3 = f'''select
                    round(avg(salary_from)) "Средняя з/п от",
                    round(avg(salary_to)) "Средняя з/п до"
                from public.vacancies
             '''

In [135]:
# результат запроса
df = pd.read_sql_query(query_4_3, conn_str)
display(df)

,Средняя з/п от,Средняя з/п до
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [136]:
# текст запроса
query_4_4 = f'''select
                    schedule "Тип рабочего графика",
                    employment "Тип трудоустройства",
                    count(*) "Количество вакансий"
                from public.vacancies
                group by schedule, employment /*группировка по типам графика и трудоустройства*/
                order by 3 desc /*сортировка по убыванию количества вакансий*/
                offset 1
                limit 1
             '''

In [137]:
# результат запроса
df = pd.read_sql_query(query_4_4, conn_str)
display(df)

,Тип рабочего графика,Тип трудоустройства,Количество вакансий
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [138]:
# текст запроса
query_4_5 = f'''select
                    experience "Требуемый опыт",
                    count(*) "Количество вакансий"
                from public.vacancies
                group by experience /*группировка по опыту работы*/
                order by 2 /*сортировка по возрастанию количества вакансий*/
             '''

In [139]:
# результат запроса
df = pd.read_sql_query(query_4_5, conn_str)
display(df)

,Требуемый опыт,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы:

- Наибольшее количество вакансий представлено в столицах и городах-миллионниках, что вполне логично.
- Для более чем половины вакансий в БД не указан размер заработной платы, что может быть связано с желанием работодателя определить уровень заработной платы соискателя по результатам собеседования. Средняя же з/п для вакансий с указанием такой информации колеблется от 71 до 110 тысяч рублей.
- Вакансии с полной занятостью составляют бОльшую часть предложений работодателей. Наиболее распространенным типом рабочего графика является "Полный день". Примерно каждая пятая вакансия подразумевает удаленный режим работы.
- Более 80% вакансий содержат требования по опыту работы от 1 года до 6 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [140]:
# текст запроса
query_5_1 = f'''(select
                    e.name "Работодатель",
                    count(*) "Количество вакансий"
                from public.employers e
                join public.vacancies v on e.id = v.employer_id /*соединение таблиц employers и vacancies по id работодателя*/
                group by e.name /*группировка по названию работодателя*/
                order by 2 desc
                limit 1)
    
                union all /*объединение результатов двух запросов*/

                (select
                    e.name,
                    count(*)
                from public.employers e
                join public.vacancies v on e.id = v.employer_id
                group by e.name
                order by 2 desc
                offset 4
                limit 1)
             '''

In [141]:
# результат запроса
df = pd.read_sql_query(query_5_1, conn_str)
display(df)

,Работодатель,Количество вакансий
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [142]:
# текст запроса
query_5_2 = f'''select 
                    a.name "Регион",
                    count(distinct e.id) "Количество работодателей",
                    count(distinct v.id) "Количество вакансий"
                from public.areas a
                left join public.employers e on a.id = e.area /*присоединение к таблице areas таблицы employers по id региона*/
                left join public.vacancies v on a.id = v.area_id /*присоединение таблицы вакансий по id региона*/
                group by a.id /*группировка по id региона*/
                having count(v.id) = 0 /*условие отсутствия вакансий в регионе*/
                order by 2 desc
                limit 1
             '''

In [143]:
# результат запроса
df = pd.read_sql_query(query_5_2, conn_str)
display(df)

,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0


        Дополнительное исследование: лидеры по количеству работодателей и вакансий

In [182]:
# текст запроса
query_5_2_add = f'''select 
                        a.name "Регион",
                        count(distinct e.id) "Количество работодателей",
                        count(distinct v.id) "Количество вакансий"
                    from public.areas a
                    left join public.employers e on a.id = e.area /*присоединение к таблице areas таблицы employers по id региона*/
                    left join public.vacancies v on a.id = v.area_id /*присоединение таблицы вакансий по id региона*/
                    group by a.id /*группировка по id региона*/
                    order by 2 desc
                    limit 5
                 '''

In [183]:
# результат запроса
df = pd.read_sql_query(query_5_2_add, conn_str)
display(df)

,Регион,Количество работодателей,Количество вакансий
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [144]:
# текст запроса
query_5_3 = f'''select
                    e.name "Работодатель",
                    count(distinct v.area_id) "Количество регионов с вакансиями"
                from public.employers e
                join public.vacancies v on e.id = v.employer_id /*соединение таблиц employers и vacancies по id работодателя*/
                group by e.name /*группировка по названию работодателя*/
                order by 2 desc
             '''


In [145]:
# результат запроса
df = pd.read_sql_query(query_5_3, conn_str)
display(df)

,Работодатель,Количество регионов с вакансиями
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [146]:
# текст запроса
query_5_4 = f'''select
                    count(e.id) "Количество работодателей"
                from public.employers e
                left join public.employers_industries ei on e.id = ei.employer_id /*присоединение к таблице employers таблицы employers_industries по id работодателя*/
                where ei.industry_id is null /*условие отсутствия у работодателя записи о сфере деятельности*/
             '''

In [185]:
# результат запроса
df = pd.read_sql_query(query_5_4, conn_str)
display(df)
display(f"Работодателей от общего количества в БД: {round((df.iloc[0, 0] / df_emp.iloc[0, 0]) * 100, 1)}%")

,Количество работодателей
0,8419


'Работодателей от общего количества в БД: 35.8%'

        Дополнительное исследование: количество вакансий у работодателей без указания сферы деятельности

In [148]:
# текст запроса
query_5_4_add = f'''select
                        count(v.id) "Количество вакансий"
                    from public.employers e
                    join public.vacancies v on e.id = v.employer_id /*соединение таблиц employers и vacancies по id работодателя*/
                    left join public.employers_industries ei on e.id = ei.employer_id /*присоединение таблицы employers_industries по id работодателя*/
                    where ei.industry_id is null /*условие отсутствия у работодателя записи о сфере деятельности*/
                 '''

In [149]:
# результат запроса
df = pd.read_sql_query(query_5_4_add, conn_str)
display(df)
display(f"Вакансий от общего количества в БД: {round((df.iloc[0, 0] / df_vac.iloc[0, 0]) * 100, 1)}%")

,Количество вакансий
0,10084


'Вакансий от общего количества в БД: 20.5%'

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [150]:
# текст запроса
query_5_5 = f'''select
                    e.name "Работодатель"
                from public.employers e
                join public.employers_industries ei on e.id = ei.employer_id /*соединение таблиц employers и employers_industries по id работодателя*/
                group by e.name /*группировка по названию работодателя*/
                having count(ei.industry_id) = 4 /*условие по количеству сфер деятельности*/
                order by 1
                offset 2
                limit 1
             '''

In [151]:
# результат запроса
df = pd.read_sql_query(query_5_5, conn_str)
display(df)

,Работодатель
0,2ГИС


        Дополнительное исследование: лидеры по количеству сфер деятельности

In [152]:
# текст запроса
query_5_5_add = f'''select
                        e.name "Работодатель",
                        count(ei.industry_id) "Количество сфер деятельности"
                    from public.employers e
                    join public.employers_industries ei on e.id = ei.employer_id /*соединение таблиц employers и employers_industries по id работодателя*/
                    group by e.name /*группировка по названию работодателя*/
                    order by 2 desc, 1
                 '''

In [153]:
# результат запроса
df = pd.read_sql_query(query_5_5_add, conn_str)
display(df)

,Работодатель,Количество сфер деятельности
0,Модуль,16
1,Простые решения,11
2,Прайд,10
3,Энергия,10
4,Альфа,9
...,...,...
14951,"Яркий кадр, Студия",1
14952,Яркий фотомаркет,1
14953,Ярославский завод Красный Маяк,1
14954,"Яр, отель",1


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [154]:
# текст запроса
query_5_6 = f'''select
                    count(*) "Количество работодателей"
                from public.employers_industries ei
                join public.employers e on ei.employer_id = e.id /*соединение таблиц employers_industries и employers по id работодателя*/
                join public.industries i on ei.industry_id = i.id /*присоединение таблицы industries по id сферы деятельности*/
                where i.name = 'Разработка программного обеспечения' /*условие по наименованию сферы деятельности*/
             '''

In [155]:
# результат запроса
df = pd.read_sql_query(query_5_6, conn_str)
display(df)

,Количество работодателей
0,3553


        Дополнительное исследование: наиболее популярные сферы деятельности среди работодателей

In [156]:
# текст запроса
query_5_6_add = f'''select
                        i.name "Сфера деятельности",
                        count(*) "Количество работодателей"
                    from public.employers_industries ei
                    join public.employers e on ei.employer_id = e.id /*соединение таблиц employers_industries и employers по id работодателя*/
                    join public.industries i on ei.industry_id = i.id /*присоединение таблицы industries по id сферы деятельности*/
                    group by i.name /*группировка по сфере деятельности*/
                    order by 2 desc
                    limit 5
                 '''

In [157]:
# результат запроса
df = pd.read_sql_query(query_5_6_add, conn_str)
display(df)

,Сфера деятельности,Количество работодателей
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798
4,Консалтинговые услуги,662


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [158]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
tables = pd.read_html(url) ## чтение HTML-таблиц в список из DataFrame
display(tables)

[    0                                                  1
 0 NaN  Информация в этой статье или некоторых её разд...,
      №            Город Население, тыс. чел.  (на 1 января 2023 года)   
 0    1           Москва                                        13 098  \
 1    2  Санкт-Петербург                                         5 598   
 2    3      Новосибирск                                         1 635   
 3    4     Екатеринбург                                         1 539   
 4    5           Казань                                         1 315   
 5    6  Нижний Новгород                                           NaN   
 6    7       Красноярск                                         1 197   
 7    8        Челябинск                                         1 183   
 8    9           Самара                                           NaN   
 9   10              Уфа                                           NaN   
 10  11   Ростов-на-Дону                                           Na

In [159]:
cities = tuple(tables[1].replace({'Город': {'Воронеж[a]': 'Воронеж'}})['Город'].values) # преобразование списка в кортеж
display(cities)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [160]:
# текст запроса
query_5_7 = f'''(select 
                    a.name "Город",
                    count(*) "Количество вакансий"
                from public.vacancies v
                join public.employers e on e.id = v.employer_id /*соединение таблиц vacancies и employers по id работодателя*/
                join public.areas a on a.id = v.area_id /*присоединение таблицы areas по id региона*/
                where e.name = 'Яндекс' and a.name in {cities} /*условие по наименованию работодателя и перечню городов-миллионников*/
                group by a.name /*группировка по названию города*/
                )

                union all /*объединение результатов двух запросов*/

                (select 
                    'Total',
                    count(*)
                from public.vacancies v
                join public.employers e on e.id = v.employer_id
                join public.areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {cities}
                )

                order by 2 /*сортировка итоговой таблицы*/
             '''

In [161]:
# результат запроса
df = pd.read_sql_query(query_5_7, conn_str)
display(df)
display(f"Количество строк в таблице: {df.shape[0]}")

,Город,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


'Количество строк в таблице: 17'

***

Выводы:

- Лидером по количеству вакансий в БД с большим отрывом является Яндекс - 1933 вакансий. Ростелеком, занимающий 2-е место, имеет почти в 4 раза меньше вакансий.
- Лидерами по количеству работодателей и вакансий являются Москва, Санкт-Петербург и Минск. Часть работодателей точно не указывает свое местонахождение, органичиваясь указанием региона или страны, при этом в вакансиях всегда точно указан населенный пункт. Это объясняет нулевое количество вакансий в части регионов, где есть зарегистрированные работодатели.
- Яндекс и Ростелеком являются лидерами по количеству регионов с опубликованными в них вакансиями.
- Более 35% работодателей в БД не указывают свою сферу деятельности. На долю таких работодателей приходится 10084 вакансий, что составляет чуть более 20% всех вакансий в БД (см. доп. исследование).
- Лидером по количеству сфер деятельности является компания "Модуль", указавшая 16 различных областей (см. доп. исследование).
- Разработка программного обеспечения является наиболее распространенной сферой деятельности среди работодателей в БД (см. доп. исследование).
- 25% вакансий Яндекса приходится на города-миллионники.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [197]:
# текст запроса
query_6_1 = f'''select
                    count(*) "Количество вакансий"
                from public.vacancies
                where lower(name) like '%data%' or lower(name) like '%данн%' /*условие по принадлежности вакансии к данным*/
             '''

In [200]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)
display(f"Вакансий от общего количества в БД: {round((df.iloc[0, 0] / df_vac.iloc[0, 0]) * 100, 1)}%")

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\4105223737.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,Количество вакансий
0,1771


'Вакансий от общего количества в БД: 3.6%'

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [164]:
# текст запроса
query_6_2 = f'''select 
                    count(*) "Количество вакансий"
                from public.vacancies
                where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and 
                    (name ilike '%junior%'
                    or experience ilike '%Нет опыта%'
                    or employment ilike '%Стажировка%') /*условие по критериям для junior*/
             '''

In [165]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\700150659.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,Количество вакансий
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [166]:
# текст запроса
query_6_3 = f'''select 
                    count(*) "Количество вакансий"
                from public.vacancies
                where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and 
                    (key_skills ilike '%SQL%'
                    or key_skills ilike '%postgres%') /*условие по ключевым навыкам*/
             '''

In [167]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\1206811184.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,Количество вакансий
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [168]:
# текст запроса
query_6_4 = f'''select 
                    count(*) "Количество вакансий"
                from public.vacancies
                where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and
                    key_skills ilike '%Python%' /*условие по ключевому навыку*/
             '''

In [169]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\4143830933.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,Количество вакансий
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [170]:
# текст запроса
query_6_5 = f'''select 
                    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) "Количество ключевых навыков"
                from public.vacancies
                where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and
                    key_skills is not null /*условие наличию ключевых навыков*/
             '''

In [171]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\3940953658.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,Количество ключевых навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [172]:
# текст запроса
query_6_6 = f'''select 
                    distinct experience "Опыт работы",
                    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) "Средняя з/п"
                from public.vacancies
                where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and 
                    (salary_from is not null or salary_to is not null) /*условие по наличию указаанной зарплаты*/
                    and 
                    experience = 'От 3 до 6 лет' /*условие по опыту работы*/
                group by 1
             '''

In [173]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\3373444971.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,Опыт работы,Средняя з/п
0,От 3 до 6 лет,243115.0


        Дополнительное исследование: распределение вакансий DS по опыту работы и з/п

In [186]:
query_6_6_1 = f'''select 
                    count(*) "Кол-во вакансий DS с указанной з/п"
                    from public.vacancies
                  where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and 
                    (salary_from is not null or salary_to is not null) /*условие по наличию указаанной зарплаты*/
               '''

In [187]:
# результат запроса
df = pd.read_sql_query(query_6_6_1, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\198901364.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6_1, connection)


,Кол-во вакансий DS с указанной з/п
0,62


In [188]:
query_6_6_2 = f'''select
                    distinct experience "Опыт работы",
                    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) "Средняя з/п",
                    count(*) "Кол-во вакансий DS с указанной з/п"
                  from public.vacancies
                  where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                    and 
                    (salary_from is not null or salary_to is not null) /*условие по наличию указаанной зарплаты*/
                  group by 1
               '''
 

In [189]:
# результат запроса
df = pd.read_sql_query(query_6_6_2, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\410717872.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6_2, connection)


,Опыт работы,Средняя з/п,Кол-во вакансий DS с указанной з/п
0,Нет опыта,74643.0,7
1,От 1 года до 3 лет,139675.0,28
2,От 3 до 6 лет,243115.0,27


In [192]:
query_6_6_3 = f'''(select
                    distinct experience "Опыт работы",
                    count(*) "Количество вакансий DS"
                  from public.vacancies
                  where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%') /*условие по принадлежности вакансии к DS*/
                  group by 1
                  )
                  
                  union all /*объединение результатов двух запросов*/
                  
                  (select
                    'Итого вакансий DS',
                    count(*)
                  from public.vacancies
                  where
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%')
                  )
                  
                  order by 2 /*сортировка итоговой таблицы*/
               '''
 

In [193]:
# результат запроса
df = pd.read_sql_query(query_6_6_3, connection)
display(df)

C:\Users\kkaza\AppData\Local\Temp\ipykernel_1796\673847767.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6_3, connection)


,Опыт работы,Количество вакансий DS
0,Нет опыта,28
1,Более 6 лет,34
2,От 3 до 6 лет,195
3,От 1 года до 3 лет,223
4,Итого вакансий DS,480


***

Выводы:

- В БД содержится 1771 вакансия, связанная с данными, что составляет 3,6% от общего количества вакансий. Учитывая, что данные в БД не самые свежие, можно преположить, что на текущий момент % таких вакансий на рынке труда выше.
- Общее количество вакансий для DS составляет 480, при этом только 51 из них для junior-специалистов, что свидетельствует о заинтересованности работодателей в опытных сотрудниках. Cреднее количество ключевых навыков в вакансиях (более 6) также косвенно подтверждает этот тезис. Распределение количества вакансий в зависимости от опыта работы показывает, что наиболее востребованы кандидаты с опытом работы от 1 года и выше.
- Из 480 вакансий DS всего для 62 указан уровень зарплаты. Примечательно, что для соискателей с опытом работы более 6 лет, значение зарплаты не было указано ни в одной из вакансий. Можно сделать вывод, что в большинстве случаев предложение о зарплате формируется по результатам собеседования c кандидатами.

# Общий вывод по проекту

В БД содержится достаточное количество вакансий в различных сферах деятельности и предметных областях. География предложений работодателей обширная, включает в себя вакансии не только на территории России, но и в сопредельных государствах. Описание вакансий полное, что позволяет проводить анализ на основе названия вакансии, ключевых навыков, опыта работы, заработной платы, типа трудоустройства и графика работы (аналогично анализу вакансий для DS). Учитывая, что бОльшая часть работодателей в БД относится к сферам разработки ПО, системной интеграции и интернет-технологий, можно проводить детальный анализ вакансий для широкого круга IT-профессий.